## Scrape Athlete List

This file pulls lists of athletes and their schools from www.athletic.net for a particular event.

Another ipynb file will be used to take this list of athletes and pull their detailed information for each event.


In [1]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from time import sleep

In [2]:
# Select the Event
# will be used to determine the url to scrape

#Event = "&Event=21" #girl 400
#Event_file = 'girl400'
#Event = "&Event=3"   #boy 400
#Event_file = 'boy400'

#Event = "&Event=53" #girl 1600
#Event_file = 'boy1600'
Event = "&Event=52" #boy 1600
Event_file = 'boy1600'

#Event = "&Event=36"  #girl Triple Jump
#Event_file = 'girlTripleJump'

Event= "&Event=19"
Event_file = 'women100'

In [12]:
# page number ranges each year to pull data
# pulls data from all districts and subdistricts (easier to pull duplicates and remove later)
# will be used to determine the url to scrape


district_ranges = [(115902, 115936), 
                   (97851, 97885), 
                   (89004, 89038), 
                   (79755, 79789),
                   (70553, 70586),
                   (59529, 59562),
                   (49416, 49448),
                   (43150, 43182),
                   (32345, 32377), 
                   (28032, 28064), 
                   (17269, 17301),
                   (12573, 12604),
                   (9316, 9347)]
                                    

In [13]:
all_athlete_list = pd.DataFrame()
for district_range in district_ranges:
    print("starting new year")
    for d in range(district_range[0], district_range[1] + 1):
        new_list = read_div_year(0,str(d),Event)
        all_athlete_list = all_athlete_list.append(new_list)

starting new year
starting new year
Encountered error invalid literal for int() with base 10: 'd><td class="text-nowrap hidden-print' for athlete
Gabrielle Thomas
starting new year
Encountered error invalid literal for int() with base 10: '<td class="text-nowrap hidden-print' for athlete
Deajah Stevens
starting new year
starting new year
starting new year
starting new year
starting new year
starting new year
starting new year
starting new year
starting new year
starting new year


In [5]:
# loop through the HTML pages for each district (and subdistricts) and
# pull the athlete names, school, etc.

# it's easier to loop districts and subdistricts but this does produce duplicate
# data which is removed later

def read_div_year(Div,Div_id,Event):
    id_key_string =     "../Athlete.aspx?AID="
    school_key_string = "../School.aspx?SchoolID="

    athlete_list_url="https://www.athletic.net/TrackAndField/Division/Event.aspx?DivID="+Div_id+Event+'&type=1&filter=24'

    response = requests.get(athlete_list_url)
    page = response.text
    soup = BeautifulSoup(page,"lxml")
    
    cols = ['ID','Name','School_ID','School','Division']
    athlete_list = pd.DataFrame(columns=cols)

    tables=soup.find_all("table")
     
    for i in tables:
        for row in i:
            try:
                str_row = str(row)
                id_index = str_row.find(id_key_string)
                if id_index != -1:
                    # extract the athlete ID
                    str_row = str_row[id_index+20:]            
                    id_str = str_row[0:8]
                    id_str = id_str.replace('\"',"")
                    id_str = id_str.replace('>','')
                    #ID = int(id_str)
                    # extract the athlete Name
                    name_index = str_row.find(">") + 1
                    str_row = str_row[name_index:]
                    name_index = str_row.find("<")
                    Name = str_row[0:name_index] # save this data
                    # extract the athlete school
                    school_index = str_row.find(school_key_string)
                    str_row = str_row[school_index+24:]
                    school_index = str_row.find('\">')
                    school_id = int(str_row[0:school_index]) # save this data
                    str_row = str_row[school_index+2:]
                    school_index = str_row.find("<")
                    school_name = str_row[0:school_index] #save this data
                    if id_str.isdigit():
                        ID = int(id_str)
                        athlete_list = athlete_list.append({'ID': ID, 'Name': Name, 'School_ID': school_id, 'School': school_name, 'Division': Div}, ignore_index=True)
            except ValueError as ve:
                print(f'Encountered error {ve} for athlete')
                print(Name)
    athlete_list = athlete_list.set_index(['ID'])
    return athlete_list


In [14]:
all_athlete_list.head()

,Name,School_ID,School,Division
ID,,,,
14390842,Chidinma Matthew,20591,Albany,0
12617904,Mariam Magassouba,20591,Albany,0
12539618,Marion Osei-Boateng,20591,Albany,0
16197839,Casey Pedone,21430,Stony Brook,0
12540844,Njaure Ewa,20661,Binghamton,0


In [15]:
all_athlete_list.shape

(2526, 4)

In [16]:
# write to file
all_athlete_list.to_csv('../athlete_list_'+Event_file+'.csv')